In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator, MaxNLocator)

In [ ]:
# Stores paths to repository root, data, and generated plots
PATH = os.getcwd() 
ROOT = PATH[0:-16]
DATA = f'{ROOT}/data_validation/data/csv'
CONVERTED=f'{DATA}/converted'
PLOTS = f'{ROOT}/images'
ORIGINAL = f'{PLOTS}/Original_Days'
REPRODUCED = f'{PLOTS}/Reproduced_Days'

In [ ]:
# Days that data was reported in April 2017
days = ['095', '096', '100', '101']

# Each telescope from the data 
telescopes = ['AA', 'LM', 'AP', 'AZ', 'SM', 'JC', 'PV']

# Stores Telescope Baselines in a dictionary to keep track of colors associated with each pair
telescopesColorsDict = {'AA_LM': '#0A5286', 
                        'AA_AP': '#000000', 
                        'AA_AZ': '#2FC527', 
                        'AA_SM': '#BE7FFF', 
                        'AA_JC': '#BE7FFF', 
                        'AA_PV': '#2A82FF', 
                        'LM_SM': '#BD00BF', 
                        'LM_PV': '#855327', 
                        'AP_LM': '#0A5286', 
                        'AP_AZ': '#2FC527', 
                        'AP_SM': '#BE7FFF', 
                        'AP_JC': '#BE7FFF', 
                        'AP_PV': '#2A82FF', 
                        'AZ_LM': '#2500FF', 
                        'AZ_SM': '#BE8706', 
                        'AZ_JC': '#BE8706', 
                        'AZ_PV': '#A3A606', 
                        'JC_LM': '#BD00BF',
                        'JC_SM': '#656565', 
                        'JC_PV': '#C6A478', 
                        'PV_SM': '#C6A478'
                       }

# **Functions to Convert Original, Processed EHT Data**

In [ ]:
def convertData(oldFile, day, base):
    
    # Save high and low images to these files
    if base=='High':
        converted = f'{CONVERTED}/Day_{day}_2017_HIGH.csv'
    else: 
        converted = f'{CONVERTED}/Day_{day}_2017_LOW.csv'
    
    # Reads these 4 columns from the csv files & stores into DataFrames
    oldData = pd.read_csv(oldFile, skiprows=[0], usecols=['T1', 'T2', 'U(lambda)', 'V(lambda)'])

    # Stores size of columns
    U_size = oldData[oldData.columns[0]].count()

    # Updates each value in the U and V columns to converted number (Giga lambda)
    for i in range(U_size) : 
        oldVal = oldData.iloc[[i],[2, 3]]
        newVal = oldVal / 1000000000
        oldData.iloc[[i],[2, 3]] = newVal
    
    # \u03BB - lambda in unicode
    # New column names 
    newCols=['T1', 'T2', 'U (G\u03BB)', 'V (G\u03BB)']

    # Export updated data to a new csv file
    oldData.to_csv(converted, index=False, header=newCols)
    
    return converted

In [ ]:
def conversionProcess(day, base):
    
    # Path to original project data for high and low frequencies for each day
    if base=="High":
        BASE = f'{DATA}/SR1_M87_2017_{day}_hi_hops_netcal_StokesI.csv'
    else:
        BASE = f'{DATA}/SR1_M87_2017_{day}_lo_hops_netcal_StokesI.csv'
    
    # Converts data & saves to new csv files
    print("Converting Day {} - {}...".format(day, base))
    new = convertData(BASE, day, base)
    
    return pd.read_csv(new)

# **Functions to plot original or converted data into...**
#### 1. High and Low frequencies for each day (8 plots total)
#### 2. Combined high and low frequencies for each day (4 plots total)
#### 3. Replica of Paper IV's Figure 1 subplot (1 plot total)

#### **_These options are here for anyone who wants to view the data on plots at refined or coarse levels._**

## **1. High and Low Frequencies for each day (8 plots total)**

To view all 8 plots, please run the following in a new cell:
```
for i in days:
    freq(i, "High")
    freq(i, "Low")
```

In [ ]:
def plot_settings(ax, fig, date, base): 
    
    # This is for creating figure titles
    if (base == "None"): 
        ax.set_title(f"{date} Frequencies of Different Baselines")
    else:
        ax.set_title(f"{date} {base} Frequencies of Different Baselines")
    ax.set_xlabel("U (G\u03BB)")
    ax.set_ylabel("V (G\u03BB)")
    
    # Set figure size
    fig.set_figheight(7)
    fig.set_figwidth(7)
    
    # Plots the baseline lengths corresponding to fringe spacings
    circle1 = plt.Circle((0, 0), radius=8.1, color="grey", fill=False, linestyle='--', linewidth='3.0', label='25 \u03BCas')
    circle2 = plt.Circle((0, 0), radius=4, color="grey", fill=False, linestyle='--', linewidth='3.0', label='50 \u03BCas')
    ax.grid(which='major', linestyle='-', linewidth='0.5', color='grey')

    # Adds axes (inverses the x axis)
    ax.set_xlim([10, -10])
    ax.set_ylim([-10, 10])

    # Adds ticks and grid lines
    ax.xaxis.set_major_locator(MultipleLocator(5))
    ax.xaxis.set_minor_locator(MultipleLocator(1))
    ax.yaxis.set_major_locator(MultipleLocator(5))
    ax.yaxis.set_minor_locator(MultipleLocator(1))

    # Adds and labels baseline lengths in the plot
    ax.add_patch(circle1)
    ax.add_patch(circle2)
    ax.text(4.8, 7,'25 \u03BCas', rotation=30)
    ax.text(3.5, 3, '50 \u03BCas', rotation=30)

    return

In [ ]:
def plot(df, fig, ax):
    
    index = 0

    # Iterates through each Telescope-Telescope (Baseline) pair
    for i in telescopes: 
        for j in telescopes: 
            if (i != j): 
                
                # Finds the baseline pair and stores it into new DataFrame
                baseline = df[df['T1'].str.contains(i) & df['T2'].str.contains(j)]
                  
                # Plots the baseline pair if it exists
                if baseline.shape[0] != 0:
                    base = f"{i}_{j}"
                    uh = baseline["U (G\u03BB)"]
                    vh = baseline["V (G\u03BB)"]
                    inverse_uh = -1 * baseline["U (G\u03BB)"]
                    inverse_vh = -1 * baseline["V (G\u03BB)"]
                    
                    ax.scatter(uh, vh, s=30, label=base, marker='o', c=telescopesColorsDict[base])
                    ax.scatter(inverse_uh, inverse_vh, s=30, label=base, marker='o', c=telescopesColorsDict[base])
                    
                index = index + 1
    return 

In [ ]:
def freq(day, base, showPlot=False):
    
    if not (os.path.isdir(ORIGINAL)):
        os.mkdir(ORIGINAL)
    
    # Path to original project data for high and low frequencies for each day
    if base=="High":
        BASE = f'{DATA}/SR1_M87_2017_{day}_hi_hops_netcal_StokesI.csv'
        SAVE = f'{ORIGINAL}/Original_Day_{day}_2017_HIGH.eps'
    else:
        BASE = f'{DATA}/SR1_M87_2017_{day}_lo_hops_netcal_StokesI.csv'
        SAVE = f'{ORIGINAL}/Original_Day_{day}_2017_LOW.eps'
    
    # Converts data & saves to new csv files
    print("Converting Day {} - {}...".format(day, base))
    new = convertData(BASE, day, base)

    # Read the converted data from new csv file
    df = pd.read_csv(new)

    print(f'Plotting...')
    
    # Plot the data
    fig, ax = plt.subplots()
    plot(df, fig, ax)

    # Decides proper plot settings for each day and frequency
    if i=='095':
        plot_settings(ax, fig, "April 5th, 2017", base)
    elif i=='096':
        plot_settings(ax, fig, "April 6th, 2017", base)
    elif i=='100':
        plot_settings(ax, fig, "April 10th, 2017", base)
    else: 
        plot_settings(ax, fig, "April 11th, 2017", base)
        
    # Saves plot as .jpg in images/ directory
    print("Saving figure...")
    plt.savefig(SAVE, bbox_inches='tight')
    
    # Toggle for showing plot in Notebook
    if (showPlot == False):
        plt.close()

    print("Complete!")
    return fig

In [ ]:
for i in days:
    freq(i, "High")
    freq(i, "Low")

## **2. Combined high and low frequencies for each day (4 plots total)**

To view all 4 plots, please run the following code in a new cell:
```
for i in days:
    highFreq = f'{DATA}/Converted/Day_{i}_2017_HIGH.csv'
    lowFreq = f'{DATA}/Converted/Day_{i}_2017_LOW.csv'
    
    reproducePlotForDay(i, highFreq, lowFreq)
 ```
 
 _**NOTE: Please make sure data has been converted before running code!**_

In [ ]:
def reproducePlotForDay(day, highFreq, lowFreq, showPlot=False):
      
    # Reads the high and low frequency csv files for the day & stores them into DataFrames
    high = pd.read_csv(highFreq)
    low = pd.read_csv(lowFreq)
    
    # Plots the data
    fig2, ax2 = plt.subplots()
    plot(high, fig2, ax2)
    plot(low, fig2, ax2)
    
    if day=='095':
        plot_settings(ax2, fig2, "April 5th, 2017", base="None")
    elif day=='096':
        plot_settings(ax2, fig2, "April 6th, 2017", base="None")
    elif day=='100':
        plot_settings(ax2, fig2, "April 10th, 2017", base="None")
    else: 
        plot_settings(ax2, fig2, "April 11th, 2017", base="None")
        
    # Saves the reproduced plots as .jpgs in images/ directory (.png or .eps instead of .jpg)
    if not(os.path.isdir(REPRODUCED)):
        os.mkdir(REPRODUCED)
    SAVE = f'{REPRODUCED}/Reproduced_Day_{day}_Frequencies.eps'
    print("Saving figure...")
    plt.savefig(SAVE, bbox_inches='tight')
    
    # Toggle for showing plot in Notebook
    if (showPlot == False):
        plt.close()

    print("Complete!")
    return fig2

In [ ]:
for i in days:
    highFreq = f'{CONVERTED}/Day_{i}_2017_HIGH.csv'
    lowFreq = f'{CONVERTED}/Day_{i}_2017_LOW.csv'

    print("Plotting Day {}...".format(i))
    reproducePlotForDay(i, highFreq, lowFreq)

## **3. Replica of Paper IV's Figure 1 subplot (1 plot total)**

In [ ]:
def subplotSettings(ax2, fig2):

    # Set figure size
    fig2.set_figheight(10)
    fig2.set_figwidth(20)

    # Sets up axes, tick marks, lines, and alignemnts for each plot
    for a in ax2:
        a.set_xlim(xmin=10, xmax=-10)
        a.set_ylim(ymin=-10, ymax=10)

        a.tick_params(top=True, right=True, bottom=True, left=True, width=1.5, direction='in', which='both', labelsize='16')

        a.xaxis.set_minor_locator(MultipleLocator(1))
        a.xaxis.set_major_locator(MaxNLocator(nbins=4, prune='lower'))

        a.yaxis.set_minor_locator(MultipleLocator(1))
        a.yaxis.set_major_locator(MaxNLocator(nbins=4, prune='lower'))
        a.set_aspect('equal')

        # Plots the baseline lengths corresponding to fringe spacings
        circle1 = plt.Circle((0, 0), radius=8.1, color="grey", fill=False, linestyle='--', linewidth='2.5', label='25 \u03BCas')
        circle2 = plt.Circle((0, 0), radius=4, color="grey", fill=False, linestyle='--', linewidth='2.5', label='50 \u03BCas')
        a.axhline(0, linestyle='-', linewidth='1.5', color='grey')
        a.axvline(0, linestyle='-', linewidth='1.5', color='grey')

        # Adds and labels baseline lengths in the plot
        a.add_patch(circle1)
        a.add_patch(circle2)

    plt.subplots_adjust(wspace=0)

    return

In [ ]:
def subplot(df, ax):
    
    index = 0

    # Iterates through each Telescope-Telescope (Baseline) pair
    for i in telescopes: 
        for j in telescopes: 
            if (i != j): 

                # Finds the baseline pair and stores it into new DataFrame
                baseline = df[df['T1'].str.contains(i) & df['T2'].str.contains(j)]

                # Plots the baseline pair if it exists
                if baseline.shape[0] != 0:
                    base = f"{i}_{j}"
                    uh = baseline["U (G\u03BB)"]
                    vh = baseline["V (G\u03BB)"]
                    inverse_uh = -1 * baseline["U (G\u03BB)"]
                    inverse_vh = -1 * baseline["V (G\u03BB)"]

                    ax.scatter(uh, vh, s=30, label=base, marker='o', c=telescopesColorsDict[base])
                    ax.scatter(inverse_uh, inverse_vh, s=30, label=base, marker='o', c=telescopesColorsDict[base])

                index = index + 1                
    return 

# **Create Paper IV: Figure 1's plot of all 4 days**

To view this plot, please run the code in the cell below.

In [ ]:
# Create figure of 4 plots in 1 graph
fig2, ax2 = plt.subplots(1,4, sharey=True, squeeze=True)

# Sets up plot settings 
subplotSettings(ax2, fig2)

counter = 0
for i in days:
    # Converts the original data and saves to new .csv files
    high = conversionProcess(i, "High")
    low = conversionProcess(i, "Low")

    print("Plotting Day {}...\n".format(i))
    
    # Plots the converted data 
    subplot(high, ax2[counter])
    subplot(low, ax2[counter])
    counter = counter + 1

# Adds axis labels in the appropriate locations
ax2[2].set_xlabel("U (G\u03BB)", fontsize='18', loc='left')
ax2[0].set_ylabel("V (G\u03BB)", fontsize='18')

# Saves the reproduced plots as .jpgs in images/ directory (.png or .eps instead of .jpg)
print("Saving figure...")

SAVE = f'{PLOTS}/Reproduced_All_Days.eps'
plt.savefig(SAVE, bbox_inches='tight')
plt.close()
print("Complete!")